In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import torchvision
import torchvision.models as models
import sinabs
from torchvision import datasets, transforms
from PIL import Image
import sinabs.layers as sl
import numpy as np
import quartz
import copy
from tqdm.auto import tqdm
from quartz.utils import get_accuracy, encode_inputs, decode_outputs, plot_output_histograms, plot_output_comparison, normalize_weights, count_n_neurons, fuse_all_conv_bn
from typing import List

np.set_printoptions(suppress=True)


In [ ]:
from cifar10_models.resnet import resnet18

model = resnet18(pretrained=True)
model.eval();

In [ ]:
from cifar10_models.mobilenetv2 import mobilenet_v2

model = mobilenet_v2(pretrained=True)
# model = nn.Sequential(*model.features, nn.AvgPool2d(4), nn.Flatten(), *model.classifier)
model.eval();

In [ ]:
from cifar10_models.vgg import vgg11_bn

model = vgg11_bn(pretrained=True)
model.eval();

In [ ]:
batch_size = 128
device = 'cuda'

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2471, 0.2435, 0.2616))
])

valid_dataset = datasets.CIFAR10(root='./data', train=False, transform=transform, download=True)
valid_loader = DataLoader(dataset=valid_dataset, shuffle=False, batch_size=batch_size)
test_loader = DataLoader(dataset=valid_dataset, batch_size=1000, shuffle=True)

In [ ]:
get_accuracy(model, valid_loader, device)

In [ ]:
folded_model = copy.deepcopy(model)
fuse_all_conv_bn(folded_model)

In [ ]:
get_accuracy(folded_model, valid_loader, device)

In [ ]:
# folded_model

In [ ]:
def normalize_outputs(
    model: nn.Module,
    sample_data: torch.Tensor,
    percentile: float = 99,
    max_outputs = []
):
    def save_data(lyr, input, output):
        max_outputs.append(np.percentile(output.cpu().numpy(), percentile))

    module_input = []
    def get_module_input(module, input, output):
        module_input.append(input[0])

    for name, module in model.named_children(): # immediate children
        if list(module.named_children()): # is not empty (not a leaf)
            handle = module.register_forward_hook(get_module_input)
            with torch.no_grad():
                model(sample_data)
            sample_input = module_input[-1]
            max_outputs = normalize_outputs(module, sample_input, percentile, max_outputs)
            handle.remove()

        if isinstance(module, (nn.Conv2d, nn.Linear)):
            handle = module.register_forward_hook(save_data)

            with torch.no_grad():
                _ = model(sample_data)
                max_layer_output = max_outputs[-1]
                module.weight.data /= max_layer_output
                if hasattr(module, 'bias'):
                    bias_scale = np.product(np.array(max_outputs))
                    # print(f"weight scale: {1/max_layer_output}, bias_scale: {1/bias_scale}")
                    module.bias.data /= bias_scale
            handle.remove()
    return max_outputs

In [ ]:
norm_model = copy.deepcopy(folded_model)
sample_data = next(iter(test_loader))[0].cuda()
percentile = 99.99
normalize_outputs(norm_model, sample_data=sample_data, percentile=percentile, max_outputs=[])

In [ ]:
get_accuracy(norm_model, valid_loader, device)

In [ ]:
normalize_outputs(norm_model, sample_data=sample_data, percentile=percentile, max_outputs=[])